# RAG Post Retrieval Optimization - Reranking featuring Amazon SageMaker, Bedrock and llamaindex

- Vector Database (Faiss / local on the notebook for this demo)
- LLM (Amazon Bedrock - Claude3 Sonnet)
- Embeddings Model (Bedrock Titan Text Embedding v2.0)
- ReRanking Model (BAAI/bge-reranker-large) running on SageMaker
- Llamaindex for orchestration (ingestion, reranking, retrieval and final response synthesis)
- Datasets (Amazon SEC 10-k statements for year 2022 and 2023 )

<img src="reranking-sagemaker.png" width="800" height="400">

In [1]:
import sagemaker
import boto3
sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker role arn: arn:aws:iam::878687028155:role/sagemakerrole
sagemaker bucket: sagemaker-us-west-2-878687028155
sagemaker session region: us-west-2


In [2]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('BAAI/bge-reranker-large')
model = AutoModelForSequenceClassification.from_pretrained('BAAI/bge-reranker-large')

pairs = [['I love you', 'i like you'], ['I love you', 'i hate you']]
with torch.no_grad():
    inputs = tokenizer(pairs, padding=True, truncation=True, return_tensors='pt', max_length=512)
    scores = model(**inputs, return_dict=True).logits.view(-1, ).float()
    print(scores)


tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/801 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

tensor([ 7.9267, -6.0210])


In [3]:
!mkdir bge-reranker
!mkdir bge-reranker/code

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [4]:
%%writefile bge-reranker/code/inference.py

import json
import logging
import torch
from typing import List
from sagemaker_inference import encoder
from transformers import AutoModelForSequenceClassification, AutoTokenizer

PAIRS = "pairs"
SCORES = "scores"

class CrossEncoder:
    def __init__(self) -> None:
        self.device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
        logging.info(f"Using device: {self.device}")
        model_name = 'BAAI/bge-reranker-large'
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForSequenceClassification.from_pretrained(model_name)
        self.model = self.model.to(self.device)

    def __call__(self, pairs: List[List[str]]) -> List[float]:
        with torch.inference_mode():
            inputs = self.tokenizer(pairs, padding=True, truncation=True, return_tensors='pt', max_length=512)
            inputs = inputs.to(self.device)
            scores = self.model(**inputs, return_dict=True).logits.view(-1, ).float()

        return scores.detach().cpu().tolist()

def model_fn(model_dir: str) -> CrossEncoder:
    try:
        return CrossEncoder()
    except Exception:
        logging.exception(f"Failed to load model from: {model_dir}")
        raise

def transform_fn(cross_encoder: CrossEncoder, input_data: bytes, content_type: str, accept: str) -> bytes:
    payload = json.loads(input_data)
    model_output = cross_encoder(**payload)
    output = {SCORES: model_output}
    return encoder.encode(output, accept)


Writing bge-reranker/code/inference.py


In [5]:
%%writefile bge-reranker/code/__init__.py



Writing bge-reranker/code/__init__.py


In [6]:
%%writefile bge-reranker/code/requirements.txt

accelerate==0.24.1
bitsandbytes==0.41.2.post2
transformers==4.37.0
sentencepiece==0.1.99
protobuf==3.20.1

Writing bge-reranker/code/requirements.txt


In [10]:
!cd bge-reranker && tar zcvf BAAI_bge-reranker-large.tar.gz *

BAAI_bge-reranker-base.tar.gz
code/
code/__init__.py
code/requirements.txt
code/inference.py


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [11]:
repository = "BAAI/bge-reranker-large"
model_id=repository.split("/")[-1]
s3_location=f"s3://{sess.default_bucket()}/custom_inference/{model_id}/model.tar.gz"

In [12]:
!aws s3 cp bge-reranker/BAAI_bge-reranker-large.tar.gz $s3_location

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


upload: bge-reranker/BAAI_bge-reranker-large.tar.gz to s3://sagemaker-us-west-2-878687028155/custom_inference/bge-reranker-large/model.tar.gz


In [13]:
pip install -U sagemaker

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 26.0 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.228.0
    Uninstalling sagemaker-2.228.0:
      Successfully uninstalled sagemaker-2.228.0
Note: you may need to restart the kernel to use updated packages.


In [14]:
from sagemaker.huggingface.model import HuggingFaceModel

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
   model_data=s3_location,       # path to your model and script
   role=role,                    # iam role with permissions to create an Endpoint
   transformers_version="4.37",  # transformers version used
   pytorch_version="2.1",        # pytorch version used
   py_version='py310',            # python version used
)

# deploy the endpoint endpoint
predictor = huggingface_model.deploy(
    initial_instance_count=1,
    instance_type="ml.g5.2xlarge"
    )

--------------!

In [15]:
import boto3, json
session = boto3.Session()
client = session.client("sagemaker-runtime", region_name="us-west-2")
output = client.invoke_endpoint(EndpointName="huggingface-pytorch-inference-2024-08-15-17-12-07-999", Body="{\"pairs\": [[\"I love you\", \"i like you\"], [\"I love you\", \"i hate you\"]]}", ContentType="application/json")
json.loads(output["Body"].read().decode("utf-8"))

{'scores': [7.926665782928467, -6.021017074584961]}

In [16]:
#%pip install sagemaker
from sagemaker.huggingface.model import HuggingFacePredictor


predictor = HuggingFacePredictor(
    endpoint_name="huggingface-pytorch-inference-2024-08-15-17-12-07-999",
)

query = "What specific risks are typically highlighted in the risk factors section of a Form 10-K, and how can this section guide investment decisions?"

retrieved_documents = [
    "Financial documentation has evolved over centuries, with early records focusing on basic trade and barter systems. While this evolution is intriguing, it does not directly relate to the contemporary structure or specific contents of the risk factors section in a Form 10-K.",
    "Many companies include a section on future projections and market forecasts in their annual reports. While this provides a forward-looking perspective, it doesn't specifically address the risk factors or the detailed analysis of risks that is typical of a Form 10-K's risk factors section.",
    "The management's discussion in a Form 10-K offers insights into the company's operational strategy and financial planning. While informative, it is not as explicitly focused on the detailed risks or investment guidance as the risk factors section is.",
    "The risk factors section in a Form 10-K outlines various threats, such as changes in consumer preferences or supply chain disruptions. While it identifies risks, it might not always provide a thorough, specific analysis or direct guidance on making investment decisions based on those risks.",
    "The risk factors section of a Form 10-K explicitly highlights specific risks such as regulatory requirements, market volatility, and operational constraints. This section is pivotal in guiding investment decisions by offering a detailed examination of how each risk can potentially influence the company's financial trajectory and operational soundness."
]

# Create a pair for the query and each retrieved document
pairs = [[query, doc] for doc in retrieved_documents]

# Score the pairs
scores = predictor.predict({"pairs": pairs})["scores"]

print(scores)
# e.g. [1.349609375, 4.9765625, 6.70703125, 10.640625, 11.03125]

# Sort retrieved_documents by scores: returns List[str]
sorted_documents = [doc for doc, _ in sorted(zip(retrieved_documents, scores), key=lambda x: x[1], reverse=True)]

[-2.8626344203948975, 0.05411258339881897, 0.8370493054389954, 4.698368549346924, 9.260848045349121]


In [17]:
!pip install llama-index
%pip install llama-index-llms-bedrock
%pip install llama-index-embeddings-bedrock
!pip uninstall pydantic -y
!pip install pydantic
%pip install sqlalchemy==2.0.21 --force-reinstall --quiet
%pip install llama-index-embeddings-instructor

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


...

  Using cached pydantic-2.8.2-py3-none-any.whl.metadata (125 kB)
Using cached pydantic-2.8.2-py3-none-any.whl (423 kB)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-server 2.14.1 requires packaging>=22.0, but you have packaging 21.3 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [36]:
from typing import Any, List, Optional

from llama_index.core.bridge.pydantic import Field, PrivateAttr
from llama_index.core.callbacks import CBEventType, EventPayload
from llama_index.core.postprocessor.types import BaseNodePostprocessor
from llama_index.core.schema import MetadataMode, NodeWithScore, QueryBundle


class BGESageMakerRerank(BaseNodePostprocessor):

    endpoint_name: str = Field(description="SageMaker endpoint name.")
    top_n: int = Field(description="Top N nodes to return.")
    keep_retrieval_score: bool = Field(
        default=False,
        description="Whether to keep the retrieval score in metadata.",
    )

    _predictor: Any = PrivateAttr()

    def __init__(
        self,
        endpoint_name: str,
        top_n: int = 2,
        keep_retrieval_score: Optional[bool] = False,
    ):
        try:
            from sagemaker.huggingface.model import HuggingFacePredictor
        except ImportError:
            raise ImportError(
                "Cannot import sagemaker.huggingface.model.HuggingFacePredictor,",
                "please `pip install sagemaker`",
            )
        self._predictor = HuggingFacePredictor(
            endpoint_name=endpoint_name,
        )
        super().__init__(
            endpoint_name=endpoint_name,
            top_n=top_n,
            keep_retrieval_score=keep_retrieval_score,
        )

    @classmethod
    def class_name(cls) -> str:
        return "BGESageMakerRerank"

    def _postprocess_nodes(
        self,
        nodes: List[NodeWithScore],
        query_bundle: Optional[QueryBundle] = None,
    ) -> List[NodeWithScore]:
        if query_bundle is None:
            raise ValueError("Missing query bundle in extra info.")
        if len(nodes) == 0:
            return []

        texts = [node.node.get_content() for node in nodes]
        query = query_bundle.query_str

        # Create a pair for the query and each retrieved document
        pairs = [[query, doc] for doc in texts]
        payload = {"pairs": pairs}

        with self.callback_manager.event(
            CBEventType.RERANKING,
            payload={
                EventPayload.NODES: nodes,
                EventPayload.QUERY_STR: query_bundle.query_str,
                EventPayload.TOP_K: self.top_n,
            },
        ) as event:
            # Score the pairs
            result = self._predictor.predict(payload)
            assert "scores" in result, "No scores returned from SageMaker endpoint."
            scores = result["scores"]
            assert len(scores) == len(nodes), "Mismatch between scores and nodes."

            for node, score in zip(nodes, scores):
                if self.keep_retrieval_score:
                    # keep the retrieval score in metadata
                    node.node.metadata["retrieval_score"] = node.score
                node.score = score

            new_nodes = sorted(nodes, key=lambda x: -x.score if x.score else 0)[
                : self.top_n
            ]

            event.on_end(payload={EventPayload.NODES: new_nodes})

        return new_nodes

reranker = BGESageMakerRerank(
    endpoint_name="huggingface-pytorch-inference-2024-08-15-17-12-07-999",
    keep_retrieval_score=False,
    top_n=3,
)

In [19]:
from llama_index.embeddings.bedrock import BedrockEmbedding

In [20]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.query_engine import SubQuestionQueryEngine
from llama_index.core.callbacks import CallbackManager, LlamaDebugHandler
from llama_index.core import Settings

In [21]:
import json
from typing import Sequence, List
from llama_index.core.settings import Settings
from llama_index.llms.bedrock import Bedrock
from llama_index.embeddings.bedrock import BedrockEmbedding, Models

llm = Bedrock(model = "anthropic.claude-3-sonnet-20240229-v1:0")
embed_model = BedrockEmbedding(model = "amazon.titan-embed-text-v2:0")

Settings.llm = llm
Settings.embed_model = embed_model
Settings.chunk_size = 512

from llama_index.core.llms import ChatMessage
from llama_index.core.tools import BaseTool, FunctionTool
import nest_asyncio
nest_asyncio.apply()

In [22]:
!mkdir -p 'data/amazon/'
!wget 'https://s2.q4cdn.com/299287126/files/doc_financials/2023/q4/c7c14359-36fa-40c3-b3ca-5bf7f3fa0b96.pdf' -O 'data/amazon/amazon_2023.pdf'
!wget 'https://s2.q4cdn.com/299287126/files/doc_financials/2022/q4/d2fde7ee-05f7-419d-9ce8-186de4c96e25.pdf' -O 'data/amazon/amazon_2022.pdf'

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


--2024-08-15 17:27:47--  https://s2.q4cdn.com/299287126/files/doc_financials/2023/q4/c7c14359-36fa-40c3-b3ca-5bf7f3fa0b96.pdf
Resolving s2.q4cdn.com (s2.q4cdn.com)... 68.70.205.2, 68.70.205.1, 68.70.205.4, ...
Connecting to s2.q4cdn.com (s2.q4cdn.com)|68.70.205.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 800598 (782K) [application/pdf]
Saving to: ‘data/amazon/amazon_2023.pdf’

100%[======================================>] 800,598     --.-K/s   in 0.06s   

2024-08-15 17:27:47 (11.8 MB/s) - ‘data/amazon/amazon_2023.pdf’ saved [800598/800598]

--2024-08-15 17:27:47--  https://s2.q4cdn.com/299287126/files/doc_financials/2022/q4/d2fde7ee-05f7-419d-9ce8-186de4c96e25.pdf
Resolving s2.q4cdn.com (s2.q4cdn.com)... 68.70.205.3, 68.70.205.4, 68.70.205.1, ...
Connecting to s2.q4cdn.com (s2.q4cdn.com)|68.70.205.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 712683 (696K) [application/pdf]
Saving to: ‘data/amazon/amazon_2022.pdf’

100%[===

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [24]:
# load data
amazon_secfiles = SimpleDirectoryReader(input_dir="./data/amazon/").load_data()

# build index
amazon_index = VectorStoreIndex.from_documents(
    amazon_secfiles,
    use_async=True,
)

In [37]:
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core import QueryBundle
import pandas as pd
from IPython.display import display, HTML


pd.set_option("display.max_colwidth", -1)


def get_retrieved_nodes(
    query_str, vector_top_k=5, reranker_top_n=3, with_reranker=False
):
    query_bundle = QueryBundle(query_str)
    # configure retriever
    retriever = VectorIndexRetriever(
        index=amazon_index,
        similarity_top_k=vector_top_k,
    )
    retrieved_nodes = retriever.retrieve(query_bundle)

    if with_reranker:
        # configure reranker
        reranker = BGESageMakerRerank(endpoint_name="huggingface-pytorch-inference-2024-08-15-17-12-07-999",keep_retrieval_score=False,top_n=3)
        retrieved_nodes = reranker.postprocess_nodes(
            retrieved_nodes, query_bundle
        )

    return retrieved_nodes


def pretty_print(df):
    return display(HTML(df.to_html().replace("\\n", "")))


def visualize_retrieved_nodes(nodes) -> None:
    result_dicts = []
    for node in nodes:
        result_dict = {"Score": node.score, "Text": node.node.get_text()}
        result_dicts.append(result_dict)

    pretty_print(pd.DataFrame(result_dicts))

/tmp/ipykernel_998/2759056680.py:7: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option("display.max_colwidth", -1)


In [76]:
new_nodes = get_retrieved_nodes(
    "Describe key business risks for Amazon during covid",
    vector_top_k=10,
    with_reranker=False,
)

In [77]:
visualize_retrieved_nodes(new_nodes)

,Score,Text
0,0.731064,"These risk factors do not identify all risks that we face; our operations could also be affected by factors, events, oruncertainties that are not presently known to us or that we currently do not consider to present significant risks to our operations. In addition to the factorsdiscussed in Item 7 of Part II, “Management’s Discussion and Analysis of Financial Condition and Results of Operations—Overview,” and in the risk factorsbelow, global economic and geopolitical conditions and additional or unforeseen circumstances, developments, or events may give rise to or amplify many ofthe risks discussed below. Many of the risks discussed below also impact our customers, including third-party sellers, which could indirectly have a materialadverse effect on us.Business and Industry RisksWe Face Intense CompetitionOur businesses are rapidly evolving and intensely competitive, and we have many competitors across geographies, including cross-border competition,and in different industries, including physical, e-commerce, and omnichannel retail, e-commerce services, web and infrastructure computing services,electronic devices, digital content, advertising, grocery, and transportation and logistics services. Some of our current and potential competitors have greaterresources, longer histories, more customers, and/or greater brand recognition, particularly with our newly-launched products and services and in our newergeographic regions. They may secure better terms from vendors, adopt more aggressive pricing, and devote more resources to technology, infrastructure,fulfillment, and marketing.Competition continues to intensify, including with the development of new business models and the entry of new and well-funded competitors, and asour competitors enter into business combinations or alliances and established companies in other market segments expand to become competitive with ourbusiness. In addition, new and enhanced technologies, including search, web and infrastructure computing services, digital content, and electronic devicescontinue to increase our competition. The Internet facilitates competitive entry and comparison shopping, which enhances the ability of new, smaller, or lesserknown businesses to compete against us. As a result of competition, our product and service offerings may not be successful, we may fail to gain or may losebusiness, and we may be required to increase our spending or lower prices, any of which could materially reduce our sales and profits."
1,0.725057,"These risk factors do not identify all risks that we face; our operations could also be affected by factors, events, oruncertainties that are not presently known to us or that we currently do not consider to present significant risks to our operations. In addition to the factorsdiscussed in Item 7 of Part II, “Management’s Discussion and Analysis of Financial Condition and Results of Operations,” and in the risk factors below, globaleconomic and geopolitical conditions and additional or unforeseen circumstances, developments, or events may give rise to or amplify many of the risksdiscussed below. Many of the risks discussed below also impact our customers, including third-party sellers, which could indirectly have a material adverseeffect on us.Business and Industry RisksWe Face Intense CompetitionOur businesses are rapidly evolving and intensely competitive, and we have many competitors across geographies, including cross-border competition,and in different industries, including physical, e-commerce, and omnichannel retail, e-commerce services, web and infrastructure computing services,electronic devices, digital content, advertising, grocery, and transportation and logistics services. Some of our current and potential competitors have greaterresources, longer histories, more customers, and/or greater brand recognition, particularly with our newly-launched products and services and in our newergeographic regions. They may secure bette

In [78]:
new_nodes = get_retrieved_nodes(
    "Describe key business risks for Amazon during covid",
    vector_top_k=10,
    reranker_top_n=3,
    with_reranker=True,
)

In [79]:
visualize_retrieved_nodes(new_nodes)

,Score,Text
0,-1.702502,"We Face Significant Inventory RiskIn addition to risks described elsewhere in this Item 1A relating to fulfillment network and inventory optimization by us and third parties, we are exposedto significant inventory risks that may adversely affect our operating results as a result of seasonality, new product launches, rapid changes in product cyclesand pricing, defective merchandise, changes in customer demand and consumer spending patterns, changes in consumer tastes with respect to our products,spoilage, and other factors. We endeavor to accurately predict these trends and avoid overstocking or understocking products we manufacture and/or sell.Demand for products, however, can change significantly between the time inventory or components are ordered and the date of sale. In addition, when webegin selling or manufacturing a new product, it may be difficult to establish vendor relationships, determine appropriate product or component selection, andaccurately forecast demand. The acquisition of certain types of inventory or components requires significant lead-time and prepayment and they may not bereturnable. We carry a broad selection and significant inventory levels of certain products, such as consumer electronics, and at times we are unable to sellproducts in sufficient quantities or to meet demand during the relevant selling seasons. Any one of the inventory risk factors set forth above may adverselyaffect our operating results.We Are Subject to Payments-Related RisksWe accept payments using a variety of methods, including credit card, debit card, credit accounts (including promotional financing), gift cards, directdebit from a customer’s bank account, consumer invoicing, physical bank check, and payment upon delivery. For existing and future payment options we offerto our customers, we currently are subject to, and may become subject to additional, regulations and compliance requirements (including obligations toimplement enhanced authentication processes that could result in significant costs and reduce the ease of use of our payments products), as well as fraud. Forcertain payment methods, including credit and debit cards, we pay interchange and other fees, which may increase over time and raise our operating costs andlower profitability. We rely on third parties to provide certain Amazon-branded payment methods and payment processing services, including the processing ofcredit cards, debit cards, electronic checks, and promotional financing."
1,-2.717652,"The loss of any of our executive officers or other key employees, the failure to successfully transition key roles, or the inability to hire, train, retain,and manage qualified personnel, could harm our business.We also rely on a significant number of personnel to operate our stores, fulfillment network, and data centers and carry out our other operations. Failureto successfully hire, train, manage, and retain sufficient personnel to meet our needs can strain our operations, increase payroll and other costs, and harm ourbusiness and reputation. In addition, changes in laws and regulations applicable to employees, independent contractors, and temporary personnel could increaseour payroll costs, decrease our operational flexibility, and negatively impact how we are able to staff our operations and supplement our workforce.We are also subject to labor union efforts to organize groups of our employees from time to time. These organizational efforts, if successful, decrease ouroperational flexibility, which could adversely affect our operating efficiency. In addition, our response to any organizational efforts could be perceivednegatively and harm our business and reputation.Our Supplier Relationships Subject Us to a Number of RisksWe have significant suppliers, including content and technology licensors, and in some cases, limited or single-sources of supply, that are important toour sourcing, services, manufacturing, and any related ongoing servic

# Seeing reranking in action in final results

In [80]:
query_engine_naive = amazon_index.as_query_engine(similarity_top_k=10)

In [81]:
response = query_engine_naive.query(
    "Describe key business risks for Amazon during covid"
)

In [82]:
print(response)

Based on the context provided, some key business risks that Amazon faced during the COVID-19 pandemic include:

1. Disruptions from the public health crisis, which could impact Amazon's operations, supply chain, and ability to meet customer demand. The context mentions "disruptions from natural or human-caused disasters (including public health crises)" as a risk factor.

2. Challenges in optimizing and operating their fulfillment network and data centers due to factors like labor constraints, supply chain issues, and fluctuations in customer demand patterns during the pandemic. The context discusses the risks of "failures to adequately predict customer demand" and staffing limitations affecting productivity across their fulfillment network.

3. Potential constraints or delays from suppliers due to the pandemic's impact, which could affect Amazon's ability to procure merchandise, content, components, or services on acceptable terms or in a timely manner. The context highlights risks re

In [83]:
query_engine_rerank = amazon_index.as_query_engine(similarity_top_k=10, node_postprocessor =[reranker])

In [84]:
response = query_engine_rerank.query(
    "Describe key business risks for Amazon during covid"
)

In [85]:
print(response)

Based on the context provided, some key business risks that Amazon faced during the COVID-19 pandemic include:

1. Disruptions from the public health crisis, which could impact Amazon's operations, supply chain, and ability to meet customer demand. The context mentions "disruptions from natural or human-caused disasters (including public health crises)" as a risk factor.

2. Challenges in optimizing and operating their fulfillment network and data centers due to factors like labor constraints, supply chain issues, and fluctuations in customer demand patterns during the pandemic. The context highlights the risks of "failures to adequately predict customer demand" and staffing limitations in the fulfillment network.

3. Potential negative impacts on Amazon's supplier relationships, as the pandemic could affect suppliers' ability to provide merchandise, content, components or services on acceptable terms or in a timely manner. The context notes risks from "decisions by our current supplie